In [36]:
data = """9C0141080250320F1802104A08
"""
data = open('puzzle.data').read()

from operator import gt, lt, eq

def prod(values):
    p = 1
    for value in values:
        p *= value
    return p

class Packet:
    def __init__(self, bit_str: str):
        self.version = int(bit_str[:3], 2)
        self.type = int(bit_str[3:6], 2)
        self.subpackets = []

        if self.type == 4:
            index = 6
            value = 0
            while True:
                value = (value << 4) + int(bit_str[index + 1: index + 5], 2)
                if bit_str[index] == '0':
                    break
                index += 5
            self.value = value
            self.length = index + 5
            return
        
        if bit_str[6] == '0':
            length = int(bit_str[7:22], 2)
            index = 22
            while length:
                subpacket = Packet(bit_str[index:])
                length -= subpacket.length
                index += subpacket.length
                self.subpackets.append(subpacket)
        else:
            length = int(bit_str[7:18], 2)
            index = 18
            for _ in range(length):
                subpacket = Packet(bit_str[index:])
                index += subpacket.length
                self.subpackets.append(subpacket)
     
        self.length = index

    @classmethod
    def from_hex_str(cls, hex_str: str):
        hex_str = hex_str.strip()
        return cls(format(int(hex_str, 16), '0>%db' % (len(hex_str) * 4)))

    def op(self):
        if self.type == 4:
            return self.value
        
        op_dict = {0: sum, 1: prod, 2: min, 3: max, 5: gt, 6: lt, 7: eq}
        values = [p.op() for p in self.subpackets]
        if self.type < 4:
            return op_dict[self.type](values)
        else:
            return int(op_dict[self.type](*values))

def sum_of_versions(packet: Packet):
    if packet.subpackets:
        return packet.version + sum(sum_of_versions(p) for p in packet.subpackets)
    return packet.version

packet = Packet.from_hex_str(data)
sum_of_versions(packet)

925

In [37]:
packet.op()

342997120375